プログラムを埋めて、cifar10をvgg16のモデルの転移学習させてください。

- cifar10のデータは全て用いて良いです
- データのかさ増し、拡張はなしとします
- 重みの固定は15層目までとしましょう
- epoch数3で65%以上の精度を出しましょう

In [ ]:
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.datasets import cifar10
from keras.layers import Dense, Dropout, Flatten, Input
from keras.models import Model, Sequential
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np

# データのロードをしてください
(X_train, y_train), (X_test, y_test) = 
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#input_tensorの定義をして、vggのImageNetによる学習済みモデルを作成してください
input_tensor = 
vgg16 = 

# 特徴量抽出部分のモデルを作成しています
top_model = vgg16.output
top_model = Flatten(input_shape=vgg16.output_shape[1:])(top_model)
top_model = Dense(256, activation='sigmoid')(top_model)
top_model = Dropout(0.5)(top_model)
top_model = Dense(10, activation='softmax')(top_model)

# vgg16とtop_modelを連結してください
model = 

# 以下のfor文を完成させて、15層目までの重みを固定させてください
for layer in :
    layer.trainable =

#　学習の前に、モデル構造を確認してください
model.summary()

# コンパイルをしています
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# すでに学習済みのモデルを保存している場合、以下のように学習済みモデルを取得できます
# model.load_weights('param_vgg_15.hdf5')

# バッチサイズ32で学習を行ってください
model.fit()

# 以下の式でモデルを保存することができます
model.save_weights('param_vgg_15.hdf5')

# 精度の評価をしています
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


ヒント

モデルの作成は、転移学習のVGG16の章を参考にしてください。


#### 解答例

In [ ]:
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.datasets import cifar10
from keras.layers import Dense, Dropout, Flatten, Input
from keras.models import Model, Sequential
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np

# データのロードをしてください
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#input_tensorの定義をして、vggのImageNetによる学習済みモデルを作成してください
input_tensor = Input(shape=(32, 32, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

# 特徴量抽出部分のモデルを作成しています
top_model = vgg16.output
top_model = Flatten(input_shape=vgg16.output_shape[1:])(top_model)
top_model = Dense(256, activation='sigmoid')(top_model)
top_model = Dropout(0.5)(top_model)
top_model = Dense(10, activation='softmax')(top_model)

# vgg16とtop_modelを連結してください
model = Model(inputs=vgg16.input, outputs=top_model)

# 以下のfor文を完成させて、15層目までの重みを固定させてください
for layer in model.layers[:15]:
    layer.trainable = False

#　学習の前に、モデル構造を確認してください
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# すでに学習済みのモデルを保存している場合、以下のように学習済みモデルを取得できます
# model.load_weights('param_vgg_15.hdf5')

# バッチサイズ32で学習を行ってください
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=3)

# 以下の式でモデルを保存することができます
model.save_weights('param_vgg_15.hdf5')

# 精度の評価をしています
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


解説

転移学習のVGG16の章とは異なり、すべての層を固定するのではなく、一部の層だけ固定して学習してみました。
今回作成したmodelの19層目まではImageNetによる重みの学習が完了していますが、今回はそのうちの15層目までを固定しました。
o.o.oでは、同じ学習方法で、およそ47%の精度が出たのに対して、今回は70%近くまで精度をあげられました。ただし、実行時間はおよそ2,3倍になりました。
固定する層を一部にしたことで、学習が必要なパラメータの数が増えたため、計算量が増えた一方、精度は向上しました。

データの数が少ない時などは、このように一部の層だけを固定する方法はとても有用ですが、精度と学習時間はトレードオフの関係にあることを意識しましょう。